In [2]:
!pip install nbimporter


In [6]:
# 🎮 Импорты
import pygame
import random
import time
import psycopg2

# 📦 Классы для игры
class Point:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

class GameObject:
    def __init__(self, points, color, tile_width):
        self.points = points
        self.color = color
        self.tile_width = tile_width

class Wall(GameObject):
    def __init__(self, level, tile_width):
        super().__init__([], (255, 0, 0), tile_width)
        self.level = level
        self.load_level()

    def load_level(self):
        self.points = []
        with open(f"levels/level{self.level}.txt", "r") as f:
            for row, line in enumerate(f):
                for col, c in enumerate(line):
                    if c == '#':
                        self.points.append(Point(col * self.tile_width, row * self.tile_width))

    def next_level(self):
        self.points = []
        self.level = (self.level + 1) % 2
        self.load_level()

    def check_collision(self, head_location):
        for point in self.points:
            if point.X == head_location.X and point.Y == head_location.Y:
                return True
        return False

# 🗄️ Работа с базой данных PostgreSQL
def connect():
    return psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="hooker2007"
    )

def create_tables():
    conn = connect()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            username VARCHAR(50) UNIQUE NOT NULL
        );
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS user_score (
            id SERIAL PRIMARY KEY,
            user_id INTEGER REFERENCES users(id),
            score INTEGER DEFAULT 0,
            level INTEGER DEFAULT 1
        );
    """)

    conn.commit()
    cur.close()
    conn.close()

def get_create_user(username):
    username = username.strip().lower()
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT id FROM users WHERE username = %s", (username,))
    user = cur.fetchone()

    if user:
        user_id = user[0]
        cur.execute("SELECT score, level FROM user_score WHERE user_id = %s", (user_id,))
        score_data = cur.fetchone()
        if score_data:
            score, level = score_data
        else:
            score, level = 0, 1
            cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))
    else:
        cur.execute("INSERT INTO users (username) VALUES (%s) RETURNING id", (username,))
        user_id = cur.fetchone()[0]
        score, level = 0, 1
        cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()
    return user_id, score, level

def save_game(user_id, score, level):
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT * FROM user_score WHERE user_id = %s", (user_id,))
    existing = cur.fetchone()

    if existing:
        cur.execute("""
            UPDATE user_score
            SET score = %s, level = %s
            WHERE user_id = %s
        """, (score, level, user_id))
    else:
        cur.execute("""
            INSERT INTO user_score (user_id, score, level)
            VALUES (%s, %s, %s)
        """, (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()
    return user_id, score, level

def show_all_users():
    conn = connect()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.username, us.score, us.level
        FROM users u
        JOIN user_score us ON u.id = us.user_id
        ORDER BY us.score DESC;
    """)
    
    users = cur.fetchall()

    print("📊 Список пользователей:")
    for user in users:
        print(f"👤 {user[0]} | 🏆 Очки: {user[1]} | 🎯 Уровень: {user[2]}")
    
    cur.close()
    conn.close()

# ✨ Пример вызова
create_tables()
show_all_users()


📊 Список пользователей:


In [23]:
# --- Импорты ---
import pygame
import random
import time
import psycopg2

# --- Классы (Point, Wall, GameObject) ---
class Point:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

class GameObject:
    def __init__(self, points, color, tile_width):
        self.points = points
        self.color = color
        self.tile_width = tile_width

class Wall(GameObject):
    def __init__(self, level, tile_width):
        super().__init__([], (255, 0, 0), tile_width)
        self.level = level
        self.load_level()

    def load_level(self):
        self.points = []
        with open(f"levels/level{self.level}.txt", "r") as f:
            for row, line in enumerate(f):
                for col, c in enumerate(line.strip()):
                    if c == '#':
                        self.points.append(Point(col * self.tile_width, row * self.tile_width))

    def next_level(self):
        self.level = (self.level + 1) % 2
        self.load_level()

    def check_collision(self, head):
        return any(point.X == head.X and point.Y == head.Y for point in self.points)

# --- БД ---
def connect():
    return psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="hooker2007"
    )

def create_tables():
    conn = connect()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            username VARCHAR(50) UNIQUE NOT NULL
        );
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS user_score (
            id SERIAL PRIMARY KEY,
            user_id INTEGER REFERENCES users(id),
            score INTEGER DEFAULT 0,
            level INTEGER DEFAULT 1
        );
    """)

    conn.commit()
    cur.close()
    conn.close()

def get_create_user(username):
    username = username.strip().lower()
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT id FROM users WHERE username = %s", (username,))
    user = cur.fetchone()

    if user:
        user_id = user[0]
        cur.execute("SELECT score, level FROM user_score WHERE user_id = %s", (user_id,))
        score_data = cur.fetchone()
        score, level = score_data if score_data else (0, 1)
    else:
        cur.execute("INSERT INTO users (username) VALUES (%s) RETURNING id", (username,))
        user_id = cur.fetchone()[0]
        score, level = 0, 1
        cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()
    return user_id, score, level

def save_game(user_id, score, level):
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT * FROM user_score WHERE user_id = %s", (user_id,))
    if cur.fetchone():
        cur.execute("UPDATE user_score SET score = %s, level = %s WHERE user_id = %s", (score, level, user_id))
    else:
        cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()

# --- Игра ---
def run_game():
    pygame.init()
    TILE = 20
    WIDTH, HEIGHT = 600, 400
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()

    # Ввод имени пользователя
    username = input("Введите имя: ")
    user_id, score, level = get_create_user(username)
    wall = Wall(level, TILE)

    snake = [Point(100, 100)]
    dx, dy = TILE, 0
    food = Point(random.randint(0, WIDTH // TILE - 1) * TILE,
                 random.randint(0, HEIGHT // TILE - 1) * TILE)

    running = True
    while running:
        screen.fill((0, 0, 0))

        # Обработка событий
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                save_game(user_id, score, level)
                running = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]: dx, dy = -TILE, 0
        if keys[pygame.K_RIGHT]: dx, dy = TILE, 0
        if keys[pygame.K_UP]: dx, dy = 0, -TILE
        if keys[pygame.K_DOWN]: dx, dy = 0, TILE

        # Движение змейки
        new_head = Point(snake[0].X + dx, snake[0].Y + dy)

        if new_head.X < 0 or new_head.X >= WIDTH or new_head.Y < 0 or new_head.Y >= HEIGHT:
            print("❌ Столкновение с границей!")
            save_game(user_id, score, level)
            break

        if any(p.X == new_head.X and p.Y == new_head.Y for p in snake):
            print("❌ Змейка врезалась в себя!")
            save_game(user_id, score, level)
            break

        if wall.check_collision(new_head):
            print("❌ Столкновение со стеной!")
            save_game(user_id, score, level)
            break

        snake.insert(0, new_head)
        if new_head.X == food.X and new_head.Y == food.Y:
            score += 1
            food = Point(random.randint(0, WIDTH // TILE - 1) * TILE,
                         random.randint(0, HEIGHT // TILE - 1) * TILE)
            if score % 3 == 0:
                wall.next_level()
                level += 1
        else:
            snake.pop()

        # Отрисовка
        pygame.draw.rect(screen, (255, 255, 0), (food.X, food.Y, TILE, TILE))
        for s in snake:
            pygame.draw.rect(screen, (0, 255, 0), (s.X, s.Y, TILE, TILE))
        for w in wall.points:
            pygame.draw.rect(screen, wall.color, (w.X, w.Y, TILE, TILE))

        pygame.display.flip()
        clock.tick(10)

    pygame.quit()

# --- Запуск игры ---
create_tables()
run_game()


Введите имя:  уу


❌ Столкновение с границей!


In [25]:
# --- Импорты ---
import pygame
import random
import time
import psycopg2

# --- Классы (Point, Wall, GameObject) ---
class Point:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

class GameObject:
    def __init__(self, points, color, tile_width):
        self.points = points
        self.color = color
        self.tile_width = tile_width

class Wall(GameObject):
    def __init__(self, level, tile_width):
        super().__init__([], (255, 0, 0), tile_width)
        self.level = level
        self.load_level()

    def load_level(self):
        self.points = []
        with open(f"levels/level{self.level}.txt", "r") as f:
            for row, line in enumerate(f):
                for col, c in enumerate(line.strip()):
                    if c == '#':
                        self.points.append(Point(col * self.tile_width, row * self.tile_width))

    def next_level(self):
        self.level = (self.level + 1) % 2
        self.load_level()

    def check_collision(self, head):
        return any(point.X == head.X and point.Y == head.Y for point in self.points)

# --- БД функции ---
def connect():
    return psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="hooker2007"
    )

def create_tables():
    conn = connect()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            username VARCHAR(50) UNIQUE NOT NULL
        );
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS user_score (
            id SERIAL PRIMARY KEY,
            user_id INTEGER REFERENCES users(id),
            score INTEGER DEFAULT 0,
            level INTEGER DEFAULT 1
        );
    """)

    conn.commit()
    cur.close()
    conn.close()

def get_create_user(username):
    username = username.strip().lower()
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT id FROM users WHERE username = %s", (username,))
    user = cur.fetchone()

    if user:
        user_id = user[0]
        cur.execute("SELECT score, level FROM user_score WHERE user_id = %s", (user_id,))
        score_data = cur.fetchone()
        score, level = score_data if score_data else (0, 1)
    else:
        cur.execute("INSERT INTO users (username) VALUES (%s) RETURNING id", (username,))
        user_id = cur.fetchone()[0]
        score, level = 0, 1
        cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()
    return user_id, score, level

def save_game(user_id, score, level):
    conn = connect()
    cur = conn.cursor()

    cur.execute("SELECT * FROM user_score WHERE user_id = %s", (user_id,))
    if cur.fetchone():
        cur.execute("UPDATE user_score SET score = %s, level = %s WHERE user_id = %s", (score, level, user_id))
    else:
        cur.execute("INSERT INTO user_score (user_id, score, level) VALUES (%s, %s, %s)", (user_id, score, level))

    conn.commit()
    cur.close()
    conn.close()

# --- Игра со стартовой задержкой ---
def run_game():
    pygame.init()
    TILE = 20
    WIDTH, HEIGHT = 600, 400
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()

    # Ввод имени пользователя
    username = input("Введите имя: ")
    user_id, score, level = get_create_user(username)
    wall = Wall(level, TILE)

    # Обратный отсчёт перед стартом
    font = pygame.font.SysFont(None, 48)
    for i in range(5, 0, -1):
        screen.fill((0, 0, 0))
        countdown_text = font.render(f"Игра начнётся через {i}...", True, (255, 255, 255))
        screen.blit(countdown_text, (WIDTH // 2 - countdown_text.get_width() // 2, HEIGHT // 2))
        pygame.display.flip()
        time.sleep(1)

    # Начальные объекты
    snake = [Point(100, 100)]
    dx, dy = TILE, 0
    food = Point(random.randint(0, WIDTH // TILE - 1) * TILE,
                 random.randint(0, HEIGHT // TILE - 1) * TILE)

    running = True
    while running:
        screen.fill((0, 0, 0))

        # События
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                save_game(user_id, score, level)
                running = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]: dx, dy = -TILE, 0
        if keys[pygame.K_RIGHT]: dx, dy = TILE, 0
        if keys[pygame.K_UP]: dx, dy = 0, -TILE
        if keys[pygame.K_DOWN]: dx, dy = 0, TILE

        # Логика движения
        new_head = Point(snake[0].X + dx, snake[0].Y + dy)

        if new_head.X < 0 or new_head.X >= WIDTH or new_head.Y < 0 or new_head.Y >= HEIGHT:
            print("❌ Столкновение с границей!")
            save_game(user_id, score, level)
            break

        if any(p.X == new_head.X and p.Y == new_head.Y for p in snake):
            print("❌ Змейка врезалась в себя!")
            save_game(user_id, score, level)
            break

        if wall.check_collision(new_head):
            print("❌ Столкновение со стеной!")
            save_game(user_id, score, level)
            break

        snake.insert(0, new_head)
        if new_head.X == food.X and new_head.Y == food.Y:
            score += 1
            food = Point(random.randint(0, WIDTH // TILE - 1) * TILE,
                         random.randint(0, HEIGHT // TILE - 1) * TILE)
            if score % 3 == 0:
                wall.next_level()
                level += 1
        else:
            snake.pop()

        # Рендеринг
        pygame.draw.rect(screen, (255, 255, 0), (food.X, food.Y, TILE, TILE))
        for s in snake:
            pygame.draw.rect(screen, (0, 255, 0), (s.X, s.Y, TILE, TILE))
        for w in wall.points:
            pygame.draw.rect(screen, wall.color, (w.X, w.Y, TILE, TILE))

        pygame.display.flip()
        clock.tick(10)

    pygame.quit()

# --- Запуск ---
create_tables()
run_game()


Введите имя:  еноик


❌ Столкновение с границей!
